In [1]:
import torch
from hpobench.benchmarks.ml.tabular_benchmark import TabularBenchmark
from ConfigSpace.hyperparameters import OrdinalHyperparameter
from ConfigSpace.configuration_space import ConfigurationSpace

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [3]:
# Initialize the benchmark
benchmark = TabularBenchmark('nn', 31)

In [4]:
benchmark.configuration_space

Configuration space object:
  Hyperparameters:
    alpha, Type: Ordinal, Sequence: {9.99999993922529e-09, 7.74263710923151e-08, 5.994842240397702e-07, 4.641588930098806e-06, 3.5938137443736196e-05, 0.00027825593133457005, 0.002154434798285365, 0.01668100617825985, 0.1291549652814865, 1.0}, Default: 9.99999993922529e-09
    batch_size, Type: Ordinal, Sequence: {4.0, 6.0, 10.0, 16.0, 25.0, 40.0, 64.0, 101.0, 161.0, 256.0}, Default: 4.0
    depth, Type: Ordinal, Sequence: {1.0, 2.0, 3.0}, Default: 1.0
    learning_rate_init, Type: Ordinal, Sequence: {9.999999747378752e-06, 3.5938137443736196e-05, 0.00012915497063659132, 0.00046415888937190175, 0.0016681005945429206, 0.005994842387735844, 0.02154434658586979, 0.07742636650800705, 0.2782559394836426, 1.0}, Default: 9.999999747378752e-06
    width, Type: Ordinal, Sequence: {16.0, 25.0, 40.0, 64.0, 101.0, 161.0, 256.0, 406.0, 645.0, 1024.0}, Default: 16.0

In [5]:
valid_config = {"alpha": 9.99999993922529e-09, "batch_size": 4.0, "depth": 1.0, "learning_rate_init": 9.999999747378752e-06, "width": 16.0}
result = benchmark.objective_function(valid_config)
(result['function_value'], result['cost'])

(0.2430976430976431, 118.15647242446539)

In [6]:
# Step 1: Random Sampling
num_samples = 500  # Decide the number of samples
random_configs = benchmark.configuration_space.sample_configuration(size=num_samples)
random_configs

[Configuration(values={
   'alpha': 0.01668100617825985,
   'batch_size': 40.0,
   'depth': 1.0,
   'learning_rate_init': 0.005994842387735844,
   'width': 101.0,
 }),
 Configuration(values={
   'alpha': 4.641588930098806e-06,
   'batch_size': 10.0,
   'depth': 1.0,
   'learning_rate_init': 1.0,
   'width': 25.0,
 }),
 Configuration(values={
   'alpha': 7.74263710923151e-08,
   'batch_size': 161.0,
   'depth': 2.0,
   'learning_rate_init': 0.0016681005945429206,
   'width': 1024.0,
 }),
 Configuration(values={
   'alpha': 7.74263710923151e-08,
   'batch_size': 10.0,
   'depth': 3.0,
   'learning_rate_init': 3.5938137443736196e-05,
   'width': 64.0,
 }),
 Configuration(values={
   'alpha': 0.002154434798285365,
   'batch_size': 16.0,
   'depth': 2.0,
   'learning_rate_init': 0.0016681005945429206,
   'width': 40.0,
 }),
 Configuration(values={
   'alpha': 1.0,
   'batch_size': 25.0,
   'depth': 3.0,
   'learning_rate_init': 0.07742636650800705,
   'width': 101.0,
 }),
 Configuration(val

In [7]:
# Step 2: Evaluate Function Values
evaluated_results = [benchmark.objective_function(config.get_dictionary()) for config in random_configs]

In [8]:
# Step 3: Normalize Configurations
import numpy as np

# Assuming random_configs is a list of sampled configurations
normalized_configs = []

for config in random_configs:
    config_dict = config.get_dictionary()

    # Normalize each hyperparameter
    alpha = np.log10(config_dict['alpha']) / 8 + 1  # Log10 scale and normalize
    batch_size = np.log2(config_dict['batch_size']) / 8  # Log2 scale and normalize
    depth = (config_dict['depth'] - 1) / 2  # Scale to 0, 0.5, 1
    learning_rate_init = np.log10(config_dict['learning_rate_init']) / 5 + 1  # Log10 scale and normalize
    width = np.log2(config_dict['width']) / 10  # Log2 scale and normalize

    # Combine into a single array and append to the list
    normalized_config = [alpha, batch_size, depth, learning_rate_init, width]
    normalized_configs.append(normalized_config)

# Convert the list of normalized configurations into a PyTorch tensor
config_tensors = torch.tensor(normalized_configs)

# Now 'config_tensors' is ready to be used for GP model fitting

In [9]:
# Step 4: Prepare Data for GP Model
# Convert configurations and function values to tensors suitable for GP
# Extracting the 'function_value' from each result
function_values = [result['function_value'] for result in evaluated_results]

# Convert the list of function values into a PyTorch tensor
function_value_tensors = torch.tensor(function_values)

# Now 'function_value_tensors' is ready to be used for GP model fitting

In [16]:
# Save the tensors
torch.save(config_tensors, 'config_tensors.pt')
torch.save(function_value_tensors, 'function_value_tensors.pt')

In [17]:
torch.manual_seed(0)
from pandora_bayesopt.utils import fit_gp_model
objective_model = fit_gp_model(config_tensors, function_value_tensors, input_standardize=True)

In [19]:
def objective_function(X):
    if X.ndim == 1:
        X = X.unsqueeze(0)
    posterior_X = objective_model.posterior(X)
    objective_X = posterior_X.mean.detach()
    return objective_X

In [21]:
dim = 5
maximize = False

In [22]:
from pandora_bayesopt.utils import find_global_optimum_scipy
global_optimum_point, global_optimum_value = find_global_optimum_scipy(objective=objective_function, dim=dim, maximize=maximize)
print("global_optimum", global_optimum_point, global_optimum_value)
print()

global_optimum [0.37992141 0.66559432 1.         0.14839136 0.94587978] 0.04959473505603834



In [23]:
from botorch.acquisition import ExpectedImprovement
from pandora_bayesopt.acquisition.gittins import GittinsIndex
from pandora_bayesopt.acquisition.ei_puc import ExpectedImprovementWithCost
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from botorch.utils.sampling import draw_sobol_samples

In [24]:
seed = 42
num_iterations = 10
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])

In [25]:
torch.manual_seed(seed)
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

# Example usage with EI
EI_optimizer = BayesianOptimizer(objective=objective_function, dim=dim, maximize=maximize, initial_points=init_x, input_standardize=True)
EI_optimizer.run(num_iterations=num_iterations, acquisition_function_class=ExpectedImprovement)
EI_best_history = EI_optimizer.get_best_history()
EI_regret_history = EI_optimizer.get_regret_history(global_optimum_value)
EI_cost_history = EI_optimizer.get_cost_history()

Iteration 0, New point: [0.36494431 0.48820326 0.61290029 0.27972136 0.63856343], New value: 0.10175487449503841
Best observed value: 0.10175487449503841
Current acquisition value: tensor(0.0237)
Cumulative cost: 1.0

Iteration 1, New point: [0.31100968 0.51476991 0.6726384  0.15496806 0.77171749], New value: 0.0963333235295993
Best observed value: 0.0963333235295993
Current acquisition value: tensor(0.0171)
Cumulative cost: 2.0

Iteration 2, New point: [0.29713927 0.70543922 0.7466657  0.20318086 0.6277713 ], New value: 0.10437014124902831
Best observed value: 0.0963333235295993
Current acquisition value: tensor(0.0144)
Cumulative cost: 3.0

Iteration 3, New point: [0.15215928 0.66757723 0.63293826 0.25062592 0.79415521], New value: 0.10346920097562019
Best observed value: 0.0963333235295993
Current acquisition value: tensor(0.0127)
Cumulative cost: 4.0

Iteration 4, New point: [0.49178884 0.52805223 0.34358333 0.49746989 0.40400794], New value: 0.11144670477200745
Best observed value